## 추천 문제로 바라본 Gameplay prediction

### 맞출 문제 1 ~ 18까지를 소비자가 샀는지 안샀는지로 생각해볼 수 있다

### user-based recommendation으로 유저끼리의 성향이 비슷하면 비슷하게 정답을 맞출 것이라 생각해볼 수 있다.

### user content를 embedding하여 유저의 속성으로 하고 이를 기반으로 유사도
### 우선, user끼리의 유사도가 높을수록 정답 여부도 유사한지 맞춰보자

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_parquet('../data/raw/input/train.parquet')
target = pd.read_csv('../data/raw/input/train_labels.csv')
display(train.head())
display(target.head())

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4


,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090312455206810_q1,1
3,20090313091715820_q1,0
4,20090313571836404_q1,1


In [3]:
train.level_group.unique()

array(['0-4', '5-12', '13-22'], dtype=object)

In [7]:
target['session'] = target['session_id'].apply(lambda x: x.split('_')[0])
target['question'] = target['session_id'].apply(lambda x: x.split('_')[1][1:])
target

,session_id,correct,session,question
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1
...,...,...,...,...
424111,22100215342220508_q18,1,22100215342220508,18
424112,22100215460321130_q18,1,22100215460321130,18
424113,22100217104993650_q18,1,22100217104993650,18
424114,22100219442786200_q18,1,22100219442786200,18


In [ ]:
GP1 = '0-4' 1-3 
GP2 = '5-12' 4-13
GP3 = '13-22' 14-18

def group_session_id(df): 
    return np.stack([gp.correct.values for _,gp in df.groupby('session_id')])

GROUP_DATA = {
    GP1: {
        'targets': group_session_id(targets[(targets.q >= 1) & (targets.q <= 3)]),
        'pos_weight': 1/2
    },
    GP2: {
        'targets': group_session_id(targets[(targets.q >= 4) & (targets.q <= 13)]),
        'pos_weight': 1
    },
    GP3: {
        'targets': group_session_id(targets[(targets.q >= 14) & (targets.q <= 18)]),
        'pos_weight': 1/2   